In [14]:
# Create and store the model

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import warnings

# Ignore UserWarning from Keras and PIL
warnings.filterwarnings("ignore", category=UserWarning, module='keras')
warnings.filterwarnings("ignore", category=UserWarning, module='PIL')

# Set paths to your dataset
base_dir = 'PetImages\\PetImages1.25K'
train_dir = os.path.join(base_dir, 'Train')
validation_dir = os.path.join(base_dir, 'Validate')

# Set parameters
img_height, img_width = 150, 150
batch_size = 32

# Create ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Changed to 'categorical'
)

# Load validation data
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Changed to 'categorical'
)

# Build the CNN model
model = models.Sequential([
    layers.Input(shape=(img_height, img_width, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(2, activation='softmax')  # Changed to support multi-category classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Changed to 'categorical_crossentropy'
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=15
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

# Save the model in the recommended Keras format
model.save('multi_category_model.keras')




# Plot training & validation performance if history is defined
if history is not None:
    # Combined plot
    plt.figure(figsize=(12, 6))

    plt.plot(history.history['accuracy'], color='red', label='Training Accuracy')
    plt.plot(history.history['loss'], color='red', linestyle='--', linewidth=2, label='Training Loss')
    plt.plot(history.history['val_accuracy'], color='green', label='Validation Accuracy')
    plt.plot(history.history['val_loss'], color='green', linestyle='--', linewidth=2, label='Validation Loss')
    plt.title('Model Training and Validation')
    plt.ylabel('Accuracy/Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=1)
    plt.grid()
    plt.show()

    # Subplots
    plt.figure(figsize=(12, 4))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], color='red', label='Training Accuracy')
    plt.plot(history.history['loss'], color='red', linestyle='--', label='Training Loss')
    plt.title('Model Training')
    plt.ylabel('Accuracy/Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(linewidth=0.5)

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['val_accuracy'], color='green', label='Validation Accuracy')
    plt.plot(history.history['val_loss'], color='green', linestyle='--', label='Validation Loss')
    plt.title('Model Validation')
    plt.ylabel('Accuracy/Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(linewidth=0.5)

    plt.tight_layout()
    plt.show()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'PetImages\\PetImages1.25K\\Train'

In [ ]:
# Load the model and use it to create predictive functions
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import matplotlib.pyplot as plt

# Load the saved model
model = tf.keras.models.load_model('multi_category_model.keras')

# Set image size parameters for the model
img_height, img_width = 150, 150

# Define category labels
categories = ['Dog', 'Cat']

# Function to predict a specific new image
def predict_image(img_path):
    # Load the image
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalise

    # Display the image
    plt.imshow(img_array[0])  # Display the image
    plt.axis('off')  # Hide the axis
    plt.show()  # Show the image

    # Make the prediction
    prediction = model.predict(img_array, verbose=1)
    predicted_label = categories[np.argmax(prediction)]  # Decodes the prediction
    return predicted_label

# Test new images
# result = predict_image('TestImages\\001.jpeg')
# print(result)

def predict_images_in_folder(folder_path):
    print(f'Starting prediction for images in the {folder_path} folder...\n')

    print('Files in folder:')
    print(os.listdir(folder_path))

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpeg', '.jpg', '.png')) and not filename.startswith('.'):
            img_path = os.path.join(folder_path, filename)

            print(f'\nProcessing {filename}...')

            img = image.load_img(img_path, target_size=(img_height, img_width))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalise

            # Display the image
            plt.imshow(img_array[0])  # Display the image
            plt.axis('off')  # Hide the axis
            plt.show()  # Show the image

            prediction = model.predict(img_array, verbose=0)
            print(f'Raw prediction output for {filename}: {prediction}')

            predicted_label = categories[np.argmax(prediction)]  # Decodes the prediction
            print(f'{filename}: {predicted_label}')

    print('\nPrediction completed.')

# Test new images in the folder
# predict_images_in_folder('TestImages')

In [16]:
import os
print(os.path.exists('PetImages\\PetImages1.25K\\Train'))
print(os.path.exists('PetImages\\PetImages1.25K\\Validate'))


False
False
